# Интерфейсы

Интерфейс - контракт, по которому класс, его реализующий, предоставляет какие-то методы.

Написание кода с опорой на интерфейсы, а не на конкретные типы позволяет:
- **Переиспользовать код, абстрагируясь от реализации.** Один раз написанный алгоритм сортировки элементов, опирающийся только на интерфейс IComparable, одинаково работает как со встроенными типами, так и с вашими.
- **Подменять реализацию, в том числе во время исполнения.**
- **Сделать код более безопасным.** Объект, передаваемый по интерфейсной ссылке предоставляет только ограниченную информацию о своих возможностях.
- **Не опасаться за последствия (по сравнению с наследованием).** Так как мы не тянем за собой реализацию, не возникает проблем, как с множественным наследованием.

## 1. Правила определения интерфейсов

В интерфейсе определяются сигнатуры *экземплярных функциональных* членов класса, кроме конструкторов. 

Т.е. недопустимы
- Поля
- Конструкторы
- Что-либо статическое

Всё остальное - можно:
- Методы
- Свойства
- События
- Индексаторы

Модификатор доступа не указывается - он априори public.

In [31]:
public interface ISomethingMessy
{
    // Метод
    void Execute();
    
    // Свойство
    string Message { get; }
    
    // Индексатор
    object this[int index] { get; set; }
    
    // Событие
    event Action MyEvent;
}

Unhandled Exception: (11,31): error CS0073: Методы доступа add и remove должны иметь тело.
(11,39): error CS0073: Методы доступа add и remove должны иметь тело.

Пример из стандартной библиотеки - System.IDisposable
```
public interface IDisposable
{
    void Dispose();
}
```

## 2. Реализация интерфейсов. Наследование

In [3]:
using System.IO;

class Base : IDisposable
{
    private FileStream fileStream;
    
    // ...
    
    // public void Dispose() { fileStream.Dispose(); }
}

Unhandled Exception: (3,14): error CS0535: '"Base" не реализует член интерфейса "IDisposable.Dispose()".

In [30]:
using System.IO;

class Base : IDisposable
{
    private FileStream fileStream;
    
    // ...
    
    public void Dispose() { fileStream.Dispose(); }
}

class Derived : Base
{
    // ...
}

// Все наследники класса автоматически реализуют интерфейсы родителя.
Derived derived = new Derived();
derived is IDisposable

True

## 3. Также доступны методы класса object

In [5]:
IComparable<int> value = 3;
value.ToString()

3

In [6]:
value.GetType()

System.Int32

## 4. ~~Реализация~~ Наследование интерфейсов интерфейсами

Можно расширить интерфейс, отнаследовав от него другой интерфейс. Типы, реализующие интерфейс-ребёнок будут обязаны реализовать функционал обоих интерфейсов.

**Однако это оправдано тогда и только тогда, когда жёсткая связь допустима.**

Иначе лучше использовать несколько маленьких интерфейсов согласно **Interface Segregation Principle**.

In [7]:
public interface IVehicle
{
    void MoveTo(float x, float y, float z);
}

public interface IWheeledVehicle : IVehicle
{
    int NumOfWheels { get; }
}

public class Car : IWheeledVehicle { }


Unhandled Exception: (11,20): error CS0535: '"Car" не реализует член интерфейса "IWheeledVehicle.NumOfWheels".
(11,20): error CS0535: '"Car" не реализует член интерфейса "IVehicle.MoveTo(float, float, float)".

Пример наследования интерфейсов из стандартной библиотеки - IEnumerable

```
public interface IEnumerable<out T> : IEnumerable
{
    IEnumerator<T> GetEnumerator();
}

public interface IEnumerable
{
    IEnumerator GetEnumerator();
}
```

## 5. Явная (explicit) и неявная (implicit) реализации интерфейса

Однако можно реализовать интерфейс, не предоставив публичную реализацию методов.

Этого можно добиться, реализовав интерфейс **явно** (explicit). Такая реализация будет доступна **только по соответствующей интерфейсной ссылке**.

In [8]:
public class MyClass : IDisposable
{
    // Неявная реализация интерфейса
    // public void Dispose() { Console.WriteLine("Implicit"); }

    // Явная реализация интерфейса
    void IDisposable.Dispose() { Console.WriteLine("Explicit"); }
}

In [9]:
MyClass myClass = new MyClass();
myClass.Dispose();

Unhandled Exception: (2,9): error CS1061: "MyClass" не содержит определения "Dispose", и не удалось найти доступный метод расширения "Dispose", принимающий тип "MyClass" в качестве первого аргумента (возможно, пропущена директива using или ссылка на сборку).

In [10]:
IDisposable disposable = new MyClass();
disposable.Dispose();

Explicit


**В чём смысл?**

Можно реализовать несколько интерфейсов, содержащих несколько одинаковых по сигнатуре методов. Если они представляют одинаковый смысл то проблем не возникает - а если они в сущности разные?

С помощью явных реализаций интерфейса можно определить **разное поведение** экземпляра в зависимости от того, по какой ссылке мы вызываем интерфейсный метод.

P.S. Пример супер надуманный

In [11]:
// "Исполнитель"
public interface IExecutor
{
    void Execute();
}

In [12]:
// "Палач"
public interface IExecutioner
{
    void Execute();
}

In [13]:
public class Officer : IExecutor, IExecutioner
{
    public void Execute() { /* some boring actions */ Console.WriteLine("Job executed."); }
    
    void IExecutioner.Execute() { /* some murderous actions */ Console.WriteLine("Intruder executed."); }
}

In [14]:
Officer officer = new Officer();
officer.Execute();

Job executed.


In [15]:
IExecutor executor = officer;
executor.Execute();

Job executed.


In [16]:
IExecutioner executioner = officer;
executioner.Execute();

Intruder executed.


## 6. Обобщённые интерфейсы

Интерфейсы могут быть обобщёнными, таким образом получив все преимущества обобщений.

Из приятного: можно реализовать один и тот же интерфейс с различными параметрами типа, т.к. *как вы знаете*, обобщённые типы с разными параметрами конструируются в разные типы.

In [17]:
public class Number : IComparable<int>, IComparable<double>, IComparable<string>
{
    private int Value { get; }
    
    public Number(int number)
    {
        Value = number;
    }

    public int CompareTo(int other) 
    {
        Console.WriteLine("Hello from int");
        return Value.CompareTo(other);
    }
    
    
    public int CompareTo(double other)
    {
        Console.WriteLine("Hello from double");
        return ((double)Value).CompareTo(other);
    }
    
    public int CompareTo(string other)
    {
        Console.WriteLine("Hello from string");
        return ((double)Value).CompareTo(double.Parse(other));
    }
}

In [18]:
Number number = new Number(42);

In [19]:
number.CompareTo(13)

Hello from int


1

In [20]:
number.CompareTo(42.5)

Hello from double


-1

In [21]:
number.CompareTo("42")

Hello from string


0

Можно использовать интерфейсы в ограничениях на аргумент-тип. Если использовать несколько, то аргумент-тип должен реализовать все.

In [22]:
public void SayHello<T>(T value) where T : IComparable<int>, IDisposable
{
    Console.WriteLine("Hello!");
}

In [23]:
public class MyClass : IComparable<int> //, IDisposable
{
    public int CompareTo(int other) => throw new NotImplementedException();
    
    public void Dispose() => throw new NotImplementedException();
}

In [24]:
MyClass obj = new MyClass();
SayHello(obj)

Unhandled Exception: (2,1): error CS0311: Тип "MyClass" не может быть использован как параметр типа "T" в универсальном типе или методе "SayHello<T>(T)". Нет преобразования неявной ссылки из "MyClass" в "IDisposable".

## 7. Реализация метода интерфейса по умолчанию

Начиная с C# 8.0 можно определять реализацию методов интерфейса по умолчанию.

Такая реализация доступна только по интерфейсной ссылке

In [25]:
public interface ISummator
{
    int Sum(IEnumerable<int> values) 
    {
        int result = 0;
        foreach(var value in values)
        {
            result += value;
        }
        return result;
    }
}

In [26]:
public class MySummator : ISummator
{
}

In [27]:
MySummator mySummator = new MySummator();

mySummator.Sum(new int[]{1, 2, 3, 4, 5})

Unhandled Exception: (3,12): error CS1061: "MySummator" не содержит определения "Sum", и не удалось найти доступный метод расширения "Sum", принимающий тип "MySummator" в качестве первого аргумента (возможно, пропущена директива using или ссылка на сборку).

In [28]:
ISummator summator = new MySummator();

summator.Sum(new int[] { 1, 2, 3, 4, 5 })

15

## 8. Абстрактный класс или интерфейс?

**Абстрактный класс:**
- Является классом, а значит наследуясь от него нельзя наследоваться от других классов;
- Может определять часть состояния и поведения;
- Наследование - очень сильная связь;

Абстрактный определяет каркас для нескольких различных реализаций сущности.

**Интерфейс:**
- Класс может реализовывать сколько угодно интерфейсов;
- Определяет (в общем случае) только *что* должен делать класс, но не *как* (в общем случае);
- Реализация интерфейс - слабая связь;

Интерфейс определяет набор свойств, которыми должна обладать сущность, её некоторый обособленный функционал.